# Prediction of treatment for sickness

The goal of the project is to predict the treatment in function of the symptomn of some people that are sick.

We used the Chronic illness: symptoms, treatments and triggers dataset on kaggle

https://www.kaggle.com/flaredown/flaredown-autoimmune-symptom-tracker

This dataset is created from an app where people with chronic sickness tell if that have symptom of something and which treatement they use. 

So, what we will try to predict what the people have use as treatment if they have some symptomns. 

So essentially what should they take if they if fell some symptome. 

The dataset have also other information that should be relevent in a lot of case like exterior effect such weather, food eated, ... 

Even if theses information are usefull, for the project during the hackathon, we only used the information of the age, sex, condition and symptom to predict the treatment.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# read the data
df = pd.read_csv('fd-export.csv')

C:\Users\satyr\.conda\envs\basepython\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
0,QEVuQwEABlEzkh7fsBBjEe26RyIVcg==,NaN,NaN,NaN,2015-11-26,1069,Condition,Ulcerative colitis,0
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,30.0,male,US,2015-11-26,1069,Condition,Ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,0.0,female,CA,2017-04-28,3168,Condition,pain in left upper arm felt like i was getting...,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,0.0,female,CA,2017-04-28,3169,Condition,hip pain when gettin up,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,0.0,female,CA,2017-04-28,3170,Condition,pain in hand joints,4


# Prepare data for each "event"

As we can see, we have information in each line correspond to info that was enter (like condition, symptome treateemnt). So we will need to create a new tabel so that for each line, we have for a user in a specific date, what was his symptoms, his conditions and his treatements.

In [4]:
def cleanTrackable_type(x):
    '''
    function to tell which trackable will be keep
    '''
    if x in ['Condition','Symptom','Treatment']:
        clean = True
    else:
        clean = False
    return clean

In [5]:
#keep only the data that was related to the condition, the symptom or the treatment
df = df[[cleanTrackable_type(i) for i in df['trackable_type']]]

In [6]:
# replace the age 0 by nan
df['age'] = df.age.replace(0.0,np.nan)
df.head()

,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
0,QEVuQwEABlEzkh7fsBBjEe26RyIVcg==,NaN,NaN,NaN,2015-11-26,1069,Condition,Ulcerative colitis,0
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,30.0,male,US,2015-11-26,1069,Condition,Ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,NaN,female,CA,2017-04-28,3168,Condition,pain in left upper arm felt like i was getting...,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,NaN,female,CA,2017-04-28,3169,Condition,hip pain when gettin up,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,NaN,female,CA,2017-04-28,3170,Condition,pain in hand joints,4


In [7]:
# the average age of the data that was filled was 32
df['age'].mean()

32.77091462554299

In [8]:
#clean the age and sex data
df['age'] = df.age.replace(np.nan,32) # replace the value missing or that were by 0 by the average value
df['age'] = df['age']/50.0 # normalize the age so that it's doesn't become much bigger than 1
df['sex'] = df.sex.replace(np.nan,'doesnt_say')

# remove the data that doesn'`t have a trackable name since that doesn't give use the info we need
df=df.dropna(subset = ['trackable_name'])

In [9]:
def uni_name(s):
  '''
  function to clean the names to be uniform or to regroup some of them
  '''
  if s=='neck and shoulder pain': return 'neck pain'
  elif s=='migraines': return 'migraine'
  elif s=='dislocations': return 'subluxation'
  elif s=='right hip pain': return 'hip pain'
  elif s=='left hip pain': return 'hip pain'
  elif s=='right shoulder pain': return 'shoulder pain'
  elif s=='left shoulder pain': return 'shoulder pain'
  elif s=='physical fatigue': return 'fatigue'
  elif s=='tiredness': return 'fatigue'
  elif s=='muscle ache': return 'fatigue'
  elif s=='body aches': return 'fatigue'
  elif s=='body aching': return 'fatigue'
  elif s=='pain': return 'fatigue'
  elif s=='exhaustion': return 'fatigue'
  elif s=='muscle pain': return 'fatigue'
  elif s=='fatigue and tiredness': return 'fatigue'
  elif s=='headaches': return 'headache'
  elif s=='vomiting': return 'nausea'
  elif s=='social anxiety': return 'anxiety'
  elif s=='stress': return 'anxiety'
  elif s=='lightheadedness': return 'dizziness'
  elif s=='dizzyness': return 'dizziness'
  elif s=='middle back pain': return 'back pain'
  elif s=='body aching': return 'back pain'
  elif s=='lower abdomen pain': return 'stomach pain'
  elif s=='abdominal pain': return 'stomach pain'
  elif s=='gas': return 'bloating'
  elif s=='dry eyes': return 'dry eye'
  elif s=='chronic fatigue': return 'chronic pain'
  elif s=='muscle cramps': return 'muscle spasms'
  elif s=='right knee pain': return 'knee pain'
  elif s=='left knee pain': return 'knee pain'
  elif s=='palpatations': return 'palpitations'
  elif s=='heart palpitations': return 'palpitations'
  elif s=='heartburn': return 'acid reflux'
  elif s=='middle back pain': return 'back pain'
  elif s=='muscle twitching': return 'tremor'
  elif s=='shaking hands': return 'tremor'
  elif s=='low appetite': return 'loss of appetite'
  elif s=='decreased appetite': return 'loss of appetite'
  elif s=='low mood': return 'mental fatigue'
  elif s=='sleep disturbance': return 'sleep problems'
  elif s=='poor sleep': return 'unrefreshing sleep'
  elif s=='suicidal thoughts': return 'suicidal ideation'
  elif s=='memory loss': return 'memory problems'
  elif s=='pins and needles': return 'numbness'
  elif s=='numbness fingers': return 'numbness'
  elif s=='low motivation': return 'lack of motivation'
  elif s=='afternoon sleepiness': return 'excess daytime sleepiness'
  elif s=='mood': return 'anger'
  elif s=='sleep disturbance': return 'sleep problems'
  elif s=='cymbalta ' : return 'duloxetine'
  elif s=='levothyroxine' : return 'thyroxin'
  elif s=='omeprazole' : return 'Omeprazole'
  elif s=='plaquenil' : return 'hydroxychloroquine'
  elif s=='synthroid' : return 'thyroxin'
  elif s=='lyrica' : return 'pregabalin'
  elif s=='zofran' : return 'ondansetron'
  elif s=='wellbutrin' : return 'Bupropion'
  elif s=='celebrex' : return 'Celecoxib'
  elif s=='prozac' : return 'Fluoxetine'
  elif s=='aleve' : return 'Naproxen'
  elif s=='lexapro' : return 'Escitalopram'
  elif s=='mezavant' : return 'Mesalazine'
  elif s=='cbd' : return 'Cannabidiol'
  elif s=='adderall' : return 'Amphetamine'
  elif s=='azathioprine' : return 'Imuran'
  elif s=='xanax' : return 'Alprazolam'
  elif s=='seroquel' : return 'Quetiapine'
  elif s=='prilosec' : return 'Omeprazole'
  elif s=='wellbutrin xl' : return 'Bupropion'
  elif s=='lamictal' : return 'Lamotrigine'
  elif s=='klonopin' : return 'clonazepam'
  elif s=='cbd oil' : return 'Cannabidiol'
  elif s=='imuran' : return 'Imuran'
  elif s=='esomeprazole' : return 'Omeprazole'
  elif s=='dexamphetamine' : return 'Amphetamine'
  elif s=='cbd from hemp' : return 'Cannabidiol'
  elif s=='cbd capsules' : return 'Cannabidiol'
  elif s=='dexlansoprazole' : return 'lansoprazole'
  elif s=='cbd vape' : return 'Cannabidiol'
  elif s=='metoprolol tartrate' : return 'Metoprolol'
  elif s=='oxycodone lp' : return 'Oxycodone'
  elif s=='cbd cream' : return 'Cannabidiol'
  elif s=='klonopin prn' : return 'clonazepam'
  elif s=='quetiapine xr' : return 'Quetiapine'
  elif s=='cannabis cbd capsule 10mg' : return 'Cannabidiol'
  elif s=='puff w/ cbd' : return 'Cannabidiol'
  elif s=='lopressor' : return 'metoprolol'
  elif s=='dextroamphetamine' : return 'Amphetamine'
  elif s=='oxycodone-acetaminophen' : return 'Oxycodone'
  elif s=='omeprazole 20mg' : return 'Omeprazole'
  elif s=='prozac 20mg' : return 'Fluoxetine'
  elif s=='quetiapine modified release' : return 'Quetiapine'
  elif s=='amphetamine salts' : return 'Amphetamine'
  elif s=='prozac (morning)' : return 'Fluoxetine'
  elif s=='19:1 cbd:thc capsule' : return 'Cannabidiol'
  elif s=='klonopin odt' : return 'clonazepam'
  elif s=='cbd joint' : return 'Cannabidiol'
  elif s=='cannabis cbd oil' : return 'Cannabidiol'
  elif s=='supplement - cbd 2 x 2 + w/s pump' : return 'Cannabidiol'
  elif s=='19:1 thc:cbd vape pen' : return 'Cannabidiol'
  elif s=='cbd balm' : return 'Cannabidiol'
  elif s=='cbd:thc' : return 'Cannabidiol'
  elif s=='nano cbd' : return 'Cannabidiol'
  elif s=='lamictal 8am' : return 'Lamotrigine'
  elif s=='klonipin' : return 'clonazepam'
  elif s=='metoprolol2' : return 'Metoprolol'
  elif s=='methylphenidate er' : return 'methylphenidate'
  elif s=='ibs': return 'irritable bowel syndrome'
  elif s=='pots': return 'postural orthostatic tachycardia syndrome'
  elif s=='postural orthostatic tachycardia syndrome (pots)': return 'postural orthostatic tachycardia syndrome'
  elif s=='gerd': return 'gastroesophageal reflux disease'
  elif s=='pcos': return 'polycystic ovary syndrome'
  elif s=='ptsd': return 'post-traumatic stress disorder'
  elif s=='post-traumatic stress disorder (ptsd)': return 'post-traumatic stress disorder'
  elif s=='ocd': return 'obsessive compulsive disorder'
  else: return s


In [10]:
# clean the trackable name
df['trackable_name'] = [i.lower() for i in df['trackable_name']]
df['trackable_name'] = df['trackable_name'].apply(uni_name)
df['trackable_name'] = [i.lower() for i in df['trackable_name']]

In [11]:
#count the cccurence of the symptome, condition and treatment
SymOrder = df[df['trackable_type']=='Symptom'].groupby('trackable_name').count().sort_values(by=['user_id'],ascending=False)
CondOrder = df[df['trackable_type']=='Condition'].groupby('trackable_name').count().sort_values(by=['user_id'],ascending=False)
TreatOrder = df[df['trackable_type']=='Treatment'].groupby('trackable_name').count().sort_values(by=['user_id'],ascending=False)

# list of the condition, symptone and treatment in order of occurence
listCond = list(CondOrder.index)
listSym = list(SymOrder.index)
listTreat = list(TreatOrder.index)

# keep only the most commun condition, symptom and treatment
# so we choose to focus only on what happen more often
listCond = listCond[:150]
listSym = listSym[:150]
listTreat = listTreat[:100]

In [12]:
# most common condition, symptome and treatement
print(listCond[:10])
print(listSym[:10])
print(listTreat[:10])

['fibromyalgia', 'depression', 'anxiety', 'chronic fatigue syndrome', 'irritable bowel syndrome', 'migraine', 'fatigue', 'postural orthostatic tachycardia syndrome', 'endometriosis', 'ehlers-danlos syndrome']
['fatigue', 'headache', 'nausea', 'stomach pain', 'anxiety', 'joint pain', 'brain fog', 'dizziness', 'diarrhea', 'back pain']
['ibuprofen', 'vitamin d', 'thyroxin', 'tramadol', 'magnesium', 'vitamin d3', 'omeprazole', 'naproxen', 'multivitamin', 'amitriptyline']


In [13]:
# regroup info by visist(i.e. the same day and same userid) to have have all info about a visit
dfCondition = df[df['trackable_type']=='Condition'].groupby(['user_id','checkin_date','sex','age'])['trackable_name'].agg(size= len, setCond = set)
dfSymptom = df[df['trackable_type']=='Symptom'].groupby(['user_id','checkin_date'])['trackable_name'].agg(size= len, setSym = set)
dfTreatment = df[df['trackable_type']=='Treatment'].groupby(['user_id','checkin_date'])['trackable_name'].agg(size= len, setTreat = set)

Merge all the info about visit into a single data frame with each line corresponding to a visit

In [14]:
dfLabel  = pd.merge(dfCondition.reset_index(), dfSymptom.reset_index()[['user_id','checkin_date','setSym']], on=['user_id','checkin_date'])
dfLabel  = pd.merge(dfLabel, dfTreatment.reset_index()[['user_id','checkin_date','setTreat']], on=['user_id','checkin_date'])

In [15]:
dfLabel.head()

,user_id,checkin_date,sex,age,size,setCond,setSym,setTreat
0,QEVuQwEA+/tEquKas1s6wFVgiluL5A==,2017-02-15,female,0.62,4,"{migraine, hashimoto's thyroiditis, diabetes, ...",{headache},{metformin}
1,QEVuQwEA+/tEquKas1s6wFVgiluL5A==,2017-02-16,female,0.62,5,"{migraine, anxiety, diabetes, polycystic ovary...",{headache},"{metformin, thyroxin, lisinopril}"
2,QEVuQwEA+/zUvhAfOMgfi3kwGIyoag==,2018-03-01,female,0.64,1,{traumatic brain injury},"{noise sensitivity, right side not belonging, ...","{novalgin, gabapentin, transcranial bright lig..."
3,QEVuQwEA+/zUvhAfOMgfi3kwGIyoag==,2018-03-02,female,0.64,1,{traumatic brain injury},"{noise sensitivity, right side not belonging, ...","{novalgin, gabapentin, massage helmet, transcr..."
4,QEVuQwEA+/zUvhAfOMgfi3kwGIyoag==,2018-03-03,female,0.64,1,{traumatic brain injury},"{noise sensitivity, right side not belonging, ...","{novalgin, gabapentin, massage helmet, transcr..."


So what we will do, is to try to predict the from the sez, age, condtions and symptomn the treatment the person should have.

# Prepare data for model

Now, we need to process the data so that it's ready for the machine learning model

In [16]:
# one-hot encode the information about gender
SexVariable = pd.get_dummies(dfLabel['sex']).values

In [17]:
# create numpy array to put the info for the model
Xage = np.array(list(dfLabel['age'])).reshape((-1,1))
Xsex = SexVariable
XCond = np.zeros((dfLabel.shape[0],len(listCond)))
XSym = np.zeros((dfLabel.shape[0],len(listSym)))
XTreat = np.zeros((dfLabel.shape[0],len(listTreat)))

In [18]:
# import preprocessing function for one-hot encoding the condtion, symptome and teatment
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
le = preprocessing.LabelEncoder()
enc = OneHotEncoder(sparse=False)

In [19]:
# fill (in one-hot manner) the info about the condition
le.fit(listCond)
for i in range((dfLabel.shape[0])):
    listTEmpo = list(dfLabel.iloc[i]['setCond'])
    listTEmpo = [i for i in listTEmpo if i in listCond]
    if len(listTEmpo)!=0:
        indiceLoc = le.transform(listTEmpo)
        for j in indiceLoc:
            XCond[i,j]=1

In [20]:
# fill (in one-hot manner) the info about the symptome
le.fit(listSym)
for i in range((dfLabel.shape[0])):
    listTEmpo = list(dfLabel.iloc[i]['setSym'])
    listTEmpo = [i for i in listTEmpo if i in listSym]
    if len(listTEmpo)!=0:
        indiceLoc = le.transform(listTEmpo)
        for j in indiceLoc:
            XSym[i,j]=1

In [21]:
# fill (in one-hot manner) the info about the treatment
le.fit(listTreat)
for i in range((dfLabel.shape[0])):
    listTEmpo = list(dfLabel.iloc[i]['setTreat'])
    listTEmpo = [i for i in listTEmpo if i in listTreat]
    if len(listTEmpo)!=0:
        indiceLoc = le.transform(listTEmpo)
        for j in indiceLoc:
            XTreat[i,j]=1

In [22]:
# construct the numpy array that correspond to the input of the model
XFinal = np.concatenate((Xage, Xsex), axis=1)
XFinal = np.concatenate((XFinal, XCond), axis=1)
XFinal = np.concatenate((XFinal, XSym), axis=1)

In [23]:
# split data into train, valid and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(XFinal, XTreat, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [24]:
#we see that we have more than 33000 data in the training set
X_train.shape[0]

33294

In [25]:
# we can sum to how often each treatment appear. We see that the dataset is not reallty unbalance even if some treatment are more common.
y_train.sum(axis=0)

array([ 519., 1708.,  440.,  394.,  435.,  528.,  616.,  512., 1171.,
        352.,  761.,  429.,  643.,  583.,  570.,  521.,  439.,  726.,
        403.,  427., 1368.,  434.,  469., 1023., 1065.,  407.,  730.,
        596.,  471.,  904.,  532., 1346.,  380.,  562., 1110., 3217.,
        496.,  254.,  457.,  450.,  354.,  717.,  474.,  399., 2012.,
        407.,  312.,  573., 1171.,  323.,  559.,  283.,  390.,  501.,
        357.,  701., 1805.,  572.,  639., 1558.,  355., 1761.,  794.,
        363.,  408., 1428.,  510.,  440., 1243.,  918., 1304.,  445.,
        369.,  530.,  717.,  399., 1085., 1154.,  403.,  429.,  424.,
       2198.,  498.,  428., 2158.,  403., 1070.,  200.,  504.,  925.,
        318., 1285., 2473., 1537.,  359., 1149.,  293.,  833.,  341.,
       1350.])

# train machine learning model

Here we will train the machine learning model to decide which one give the best result

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

In [27]:
# let try random forest

clf = RandomForestClassifier()
parameters = {'criterion':('gini', 'entropy'), 'bootstrap':(True, False),'max_features':(5, 10),'n_jobs':[8]}

grid = GridSearchCV(clf, parameters, n_jobs =8)
grid.fit(X_train, y_train)

y_test_pred = grid.best_estimator_.predict(X_test)

print('accuracy is on test: ')
print(accuracy_score(y_test, y_test_pred))

print('f1 is on test: ')
print(f1_score(y_test, y_test_pred, average = 'samples'))

# jsccard score make more sence in the case of multi-labelling
print('jaccard is on test: ')
print(jaccard_score(y_test, y_test_pred, average = 'samples'))

accuracy is on test: 
0.7771548703573932
f1 is on test: 
0.6810732978276546
jaccard is on test: 
0.6644089179000965


C:\Users\satyr\.conda\envs\basepython\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\satyr\.conda\envs\basepython\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
clf  = neighbors.KNeighborsClassifier()
parameters = {'n_neighbors':(2,3,4,5)}

grid = GridSearchCV(clf, parameters, n_jobs =8)
grid.fit(X_train, y_train)

y_test_pred = grid.best_estimator_.predict(X_test)

print('accuracy is on test: ')
print(accuracy_score(y_test, y_test_pred))

print('f1 is on test: ')
print(f1_score(y_test, y_test_pred, average = 'samples'))

# jsccard score make more sence in the case of multi-labelling
print('jaccard is on test: ')
print(jaccard_score(y_test, y_test_pred, average = 'samples'))

accuracy is on test: 
0.7510861948142957
f1 is on test: 
0.6731070678776299
jaccard is on test: 
0.6549167339363762


C:\Users\satyr\.conda\envs\basepython\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\satyr\.conda\envs\basepython\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
